**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

>The function *act* first draw a random number. If this number is greater than ```epsilon```, we chose a random action amongst all the possible actions. <br>
Otherwise, we follow the policy we learned so far thanks to the ```reinforce``` function. <br><br>
The ```epsilon``` is essential for this algorithm, because it allows the algorithm to explore the environment, and avoid stopping in a local minima.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=201 # set small when debugging
epochs_test=21 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> * ```position``` is a matrix defining the cases where the rat can goes. Typically, it is a matrix where all its border (whith a two case thickness) are black and all the other cases (on the border) are white. The black cases represents cases where the rat can't go.
* ```board``` is a matrix defining where the rewards are in the map. There is four possible value for every coefficient of the ```board``` matrix :
    - 0.5: this means that we get a reward.
    - 0: nothing happens.
    - -0.5: we obtain a small malus.
    - -1: we obtain a huge malus.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        state = env.reset()
        
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            action = agent.act(state, train = False)
            prev_state = state
            state, reward, game_over = env.act(action)
            if reward>0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        if e%10==0:
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Iteration {}. Win/lose count {}/{}. Average score ({})"
              .format(e, win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [7]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Iteration 0. Win/lose count 9.0/17.0. Average score (-8.0)
Iteration 1. Win/lose count 10.5/10.0. Average score (-3.75)
Iteration 2. Win/lose count 10.5/14.0. Average score (-3.6666666666666665)
Iteration 3. Win/lose count 9.0/14.0. Average score (-4.0)
Iteration 4. Win/lose count 12.5/15.0. Average score (-3.7)
Iteration 5. Win/lose count 12.0/15.0. Average score (-3.5833333333333335)
Iteration 6. Win/lose count 13.0/13.0. Average score (-3.0714285714285716)
Iteration 7. Win/lose count 9.0/17.0. Average score (-3.6875)
Iteration 8. Win/lose count 10.5/10.0. Average score (-3.2222222222222223)
Iteration 9. Win/lose count 6.5/18.0. Average score (-4.05)
Iteration 10. Win/lose count 6.0/15.0. Average score (-4.5)
Iteration 11. Win/lose count 8.0/12.0. Average score (-4.458333333333333)
Iteration 12. Win/lose count 9.5/16.0. Average score (-4.615384615384615)
Iteration 13. Win/lose count 11.5/11.0. Average score (-4.25)
Iteration 14. Win/lose count 12.5/17.0. Average score (-4.26666666666

Iteration 114. Win/lose count 8.5/12.0. Average score (-4.673913043478261)
Iteration 115. Win/lose count 10.5/15.0. Average score (-4.672413793103448)
Iteration 116. Win/lose count 8.5/11.0. Average score (-4.653846153846154)
Iteration 117. Win/lose count 12.0/12.0. Average score (-4.614406779661017)
Iteration 118. Win/lose count 12.5/11.0. Average score (-4.563025210084033)
Iteration 119. Win/lose count 9.5/20.0. Average score (-4.6125)
Iteration 120. Win/lose count 14.0/19.0. Average score (-4.615702479338843)
Iteration 121. Win/lose count 5.0/12.0. Average score (-4.635245901639344)
Iteration 122. Win/lose count 6.5/14.0. Average score (-4.658536585365853)
Iteration 123. Win/lose count 11.0/8.0. Average score (-4.596774193548387)
Iteration 124. Win/lose count 13.5/11.0. Average score (-4.54)
Iteration 125. Win/lose count 11.0/26.0. Average score (-4.623015873015873)
Iteration 126. Win/lose count 6.5/16.0. Average score (-4.661417322834645)
Iteration 127. Win/lose count 9.0/17.0. Ave

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> $\bullet$
\begin{align*}
 Q^{\pi}(s,a) &=E_{p^{\pi}} \left[ \left.\sum_{t=0}^{\infty} \gamma^{t}r(s_{t},a_{t}) \right| s_{0}=s,a_{0}=a \right] \\
 &= E_{p^{\pi}} \left[\left. r(s,a) + \gamma \sum_{t=1}^{\infty} \gamma^{t}r(s_{t},a_{t})\right| s_{0}=s,a_{0}=a \right] \\
 &=  E_{p^{\pi}} \left[\left. r(s,a) + \gamma  E_{p^{\pi}} \left[\left. \sum_{t=0}^{\infty} \gamma^{t} r(s_{t},a_{t}) \right| s_{0}\sim p(\cdot|s,a), a_{0} \sim \pi(\cdot|s_0) \right] \right| s, a\right] \\
 &= E_{s'\sim p(\cdot | s,a), a' \sim \pi(\cdot | s')} \left[ r(s,a) + \gamma Q^{\pi}(s',a') \right]
\end{align*} <br>
 
 > $\bullet$ By definition of the optimal value function, we have 
 \begin{align*} Q^*(s,a) &= \max_{\pi} Q^{\pi}(s,a) \\
 &= \max_{\pi} E_{s'\sim p(\cdot | s,a), a' \sim \pi(\cdot | s')} \left[ r(s,a) + \gamma Q^{\pi}(s',a') \right]\\
 &= \max_{\pi} E_{s'\sim p(\cdot | s,a)} \left[ r(s,a) + \gamma E_{a' \sim \pi(\cdot | s')} Q^{\pi}(s',a') \right]\\
 &= E_{s'\sim p(\cdot | s,a)} \left[ r(s,a) + \gamma \max_{a'} Q^*(s',a') \right]
\end{align*}
 
> $\bullet$ As our aim is to find the nearest $Q$-function from $Q^*$, we aim in minimizing the the disctance between $Q^*(s,a)$ and $Q(s,a,\theta)$.  Using the previous formula, we approximate $Q^*(s,a)$ by $r(s,a) + \gamma \max_{a'}Q's',a',\theta)$.
 \begin{equation*}
 \mathcal{\theta} = E_{s'\sim p(\cdot | s,a), a' \sim \pi(\cdot | s')}  \left[\left|\left| r(s,a) + \gamma \max_{a'} Q(s',a',\theta) - Q(s,a,\theta) \right|\right|^2\right] 
 \end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>100:
            self.memory = self.memory[1:]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Final_score :'+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q = self.model.predict(s[None, :])
        return np.argmax(q[0])
        
        
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            ######## FILL IN
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i]  = s
            target_q[i] = self.model.predict(s[None,:])
            if game_over:
                target_q[i][a] = r
            else:
                ######## FILL IN
                target_q[i][a] = r + self.discount * np.max(self.model.predict(n_s[None,:]))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        
        model = Sequential()
        model.add(Flatten(input_shape = (5,5,self.n_state)))
        model.add(Dense(32, activation = 'relu'))
        model.add(Dense(4))        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train200.mp4'))

Epoch 000/200 | Loss 0.0267 | Win/lose count 5.5/3.0 (2.5)
Epoch 001/200 | Loss 0.0041 | Win/lose count 6.5/4.0 (2.5)
Epoch 002/200 | Loss 0.0109 | Win/lose count 6.0/3.0 (3.0)
Epoch 003/200 | Loss 0.1379 | Win/lose count 4.0/6.0 (-2.0)
Epoch 004/200 | Loss 0.0211 | Win/lose count 2.5/7.0 (-4.5)
Epoch 005/200 | Loss 0.0031 | Win/lose count 2.0/2.0 (0.0)
Epoch 006/200 | Loss 0.0016 | Win/lose count 4.0/1.0 (3.0)
Epoch 007/200 | Loss 0.0516 | Win/lose count 2.5/7.0 (-4.5)
Epoch 008/200 | Loss 0.0010 | Win/lose count 3.0/2.0 (1.0)
Epoch 009/200 | Loss 0.0472 | Win/lose count 2.0/2.0 (0.0)
Epoch 010/200 | Loss 0.0001 | Win/lose count 1.5/2.0 (-0.5)
Epoch 011/200 | Loss 0.0641 | Win/lose count 2.5/2.0 (0.5)
Epoch 012/200 | Loss 0.0104 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/200 | Loss 0.0095 | Win/lose count 7.5/6.0 (1.5)
Epoch 014/200 | Loss 0.0030 | Win/lose count 4.0/0 (4.0)
Epoch 015/200 | Loss 0.0040 | Win/lose count 5.5/4.0 (1.5)
Epoch 016/200 | Loss 0.0005 | Win/lose count 1.5/6.0 

Epoch 138/200 | Loss 0.0022 | Win/lose count 6.0/1.0 (5.0)
Epoch 139/200 | Loss 0.0021 | Win/lose count 9.5/4.0 (5.5)
Epoch 140/200 | Loss 0.0031 | Win/lose count 15.0/3.0 (12.0)
Epoch 141/200 | Loss 0.0027 | Win/lose count 13.5/1.0 (12.5)
Epoch 142/200 | Loss 0.0573 | Win/lose count 20.5/1.0 (19.5)
Epoch 143/200 | Loss 0.0011 | Win/lose count 14.0/3.0 (11.0)
Epoch 144/200 | Loss 0.0971 | Win/lose count 7.5/2.0 (5.5)
Epoch 145/200 | Loss 0.0006 | Win/lose count 8.0/1.0 (7.0)
Epoch 146/200 | Loss 0.0018 | Win/lose count 16.5/2.0 (14.5)
Epoch 147/200 | Loss 0.0606 | Win/lose count 18.0/3.0 (15.0)
Epoch 148/200 | Loss 0.0046 | Win/lose count 20.5/3.0 (17.5)
Epoch 149/200 | Loss 0.0013 | Win/lose count 3.5/0 (3.5)
Epoch 150/200 | Loss 0.0056 | Win/lose count 10.0/3.0 (7.0)
Epoch 151/200 | Loss 0.0016 | Win/lose count 11.5/3.0 (8.5)
Epoch 152/200 | Loss 0.0010 | Win/lose count 7.0/1.0 (6.0)
Epoch 153/200 | Loss 0.0639 | Win/lose count 8.5/1.0 (7.5)
Epoch 154/200 | Loss 0.0583 | Win/lose cou

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [11]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, (5,5), padding='same', activation = 'relu', input_shape=(5,5,self.n_state)))
        model.add(Dropout(0.2))

        model.add(Flatten())
        model.add(Dense(64, activation ='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train200.mp4'))

Epoch 000/201 | Loss 0.0016 | Win/lose count 1.5/1.0 (0.5)
Epoch 001/201 | Loss 0.0006 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/201 | Loss 0.0014 | Win/lose count 4.0/5.0 (-1.0)
Epoch 003/201 | Loss 0.0078 | Win/lose count 3.5/5.0 (-1.5)
Epoch 004/201 | Loss 0.0004 | Win/lose count 0/2.0 (-2.0)
Epoch 005/201 | Loss 0.0031 | Win/lose count 2.0/4.0 (-2.0)
Epoch 006/201 | Loss 0.0003 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/201 | Loss 0.0010 | Win/lose count 6.5/2.0 (4.5)
Epoch 008/201 | Loss 0.0002 | Win/lose count 5.5/7.0 (-1.5)
Epoch 009/201 | Loss 0.0006 | Win/lose count 4.0/2.0 (2.0)
Epoch 010/201 | Loss 0.0002 | Win/lose count 4.0/4.0 (0.0)
Epoch 011/201 | Loss 0.0004 | Win/lose count 2.5/4.0 (-1.5)
Epoch 012/201 | Loss 0.0099 | Win/lose count 5.0/5.0 (0.0)
Epoch 013/201 | Loss 0.0007 | Win/lose count 4.5/6.0 (-1.5)
Epoch 014/201 | Loss 0.0053 | Win/lose count 6.5/2.0 (4.5)
Epoch 015/201 | Loss 0.0016 | Win/lose count 5.5/7.0 (-1.5)
Epoch 016/201 | Loss 0.0034 | Win/lose count 4.0/

Epoch 138/201 | Loss 0.0221 | Win/lose count 14.0/1.0 (13.0)
Epoch 139/201 | Loss 0.0076 | Win/lose count 19.5/1.0 (18.5)
Epoch 140/201 | Loss 0.0053 | Win/lose count 9.0/1.0 (8.0)
Epoch 141/201 | Loss 0.0006 | Win/lose count 6.5/1.0 (5.5)
Epoch 142/201 | Loss 0.0211 | Win/lose count 15.0/4.0 (11.0)
Epoch 143/201 | Loss 0.0167 | Win/lose count 23.0/4.0 (19.0)
Epoch 144/201 | Loss 0.0245 | Win/lose count 27.0/4.0 (23.0)
Epoch 145/201 | Loss 0.0362 | Win/lose count 16.0/6.0 (10.0)
Epoch 146/201 | Loss 0.0009 | Win/lose count 5.5/2.0 (3.5)
Epoch 147/201 | Loss 0.0014 | Win/lose count 10.5/3.0 (7.5)
Epoch 148/201 | Loss 0.0393 | Win/lose count 22.0/5.0 (17.0)
Epoch 149/201 | Loss 0.0025 | Win/lose count 17.0/4.0 (13.0)
Epoch 150/201 | Loss 0.0025 | Win/lose count 7.0/0 (7.0)
Epoch 151/201 | Loss 0.0266 | Win/lose count 8.0/3.0 (5.0)
Epoch 152/201 | Loss 0.0003 | Win/lose count 7.5/1.0 (6.5)
Epoch 153/201 | Loss 0.0099 | Win/lose count 17.0/5.0 (12.0)
Epoch 154/201 | Loss 0.0742 | Win/lose 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Iteration 0. Win/lose count 17.5/0. Average score (17.5)
Iteration 1. Win/lose count 7.5/0. Average score (12.5)
Iteration 2. Win/lose count 14.0/2.0. Average score (12.333333333333334)
Iteration 3. Win/lose count 26.0/1.0. Average score (15.5)
Iteration 4. Win/lose count 7.5/1.0. Average score (13.7)
Iteration 5. Win/lose count 11.0/0. Average score (13.25)
Iteration 6. Win/lose count 17.0/0. Average score (13.785714285714286)
Iteration 7. Win/lose count 5.0/0. Average score (12.6875)
Iteration 8. Win/lose count 17.5/0. Average score (13.222222222222221)
Iteration 9. Win/lose count 22.0/0. Average score (14.1)
Iteration 10. Win/lose count 7.5/0. Average score (13.5)
Iteration 11. Win/lose count 8.0/1.0. Average score (12.958333333333334)
Iteration 12. Win/lose count 10.0/0. Average score (12.73076923076923)
Iteration 13. Win/lose count 10.0/0. Average score (12.535714285714286)
Iteration 14. Win/lose count 3.0/0. Average score (11.9)
Iteration 15. Win/lose count 13.0/0

In [20]:
HTML(display_videos('cnn_test20.mp4'))

In [21]:
HTML(display_videos('fc_test20.mp4'))

> After testing the two algorithms, we can see that the ConvNet has resutls far better than the Fully Connected Network. Indeed, the first one has a score of $8.38$, while the second one has a score of $5.78$.<br>
We observe that both algorithm tends to stop exploring fast, and stay in a state where they only fluctuate between two cases they already explored. <br>
We can see that when the temperature is low, the agent tends not to move : it stays where it started and hardly move. In the case of a high temperature, the agent explore far more the map, in order to take the bonus case. It should also be noticed that the difference of performance is bigger when the temperature is low.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [12]:
from tqdm import tqdm_notebook as tqdm

def train_explore(agent,env,epoch,prefix=''):
    score = 0
    loss = 0
    
    for e in tqdm(range(epoch)):
        # For each period, we restart a fresh game, and upadte to zero the count of win and lose.
        state = env.reset()
        game_over = False
        
        win = 0
        lose = 0
        
        # We change decrease the value of epsilon with each iteration. 
        # We begin with an epsilon of 0.33.
        agent.set_epsilon(1/(e+3))
        
        while not game_over:
            # The agent performs his next acction accordingly to its policy.
            action = agent.act(state, train = True)
            
            # We get the next step, the reward, and we know whether the game is finished or not.
            prev_state = state
            state, reward, explore_reward, game_over = env.act(action)
            
            # We update the wind and lose counters.
            if reward >0:
                win = win + reward
            if reward < 0:
                lose = lose-reward
            
            # We apply the reinforcement strategy to the agent.
            loss = agent.reinforce(prev_state, state, action, explore_reward, game_over)
         
        # We save a tenth of the runs as a mp4 file.
        if e%10 == 0:
            env.draw(prefix+str(e))
        
        # We update the stats
        score += win-lose
        
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.1f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Final_score :'+str(score/epoch))      
    
def test_explore(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        state = env.reset()
        
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            action = agent.act(state, train = False)
            prev_state = state
            state, reward, reward_explore, game_over = env.act(action)
            if reward>0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Iteration {}. Win/lose count {}/{}. Average score ({})"
              .format(e, win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.alpha = 0.1

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size, grid_size)) # a board explaining whether the cases were explored.

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        explore_reward = self.board[self.x, self.y] - self.alpha * self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0
        self.malus_position[self.x, self.y] = 1
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1), 
                        self.malus_position.reshape(self.grid_size, self.grid_size, 1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, explore_reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.t = 0

        state = np.concatenate((
            self.board.reshape(self.grid_size, self.grid_size,1),
            self.position.reshape(self.grid_size, self.grid_size,1),
            self.malus_position.reshape(self.grid_size, self.grid_size, 1)), axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [14]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore200.mp4'))

Epoch 000/201 | Loss 0.0163 | Win/lose count 8.5/12.0 (-3.5)
Epoch 001/201 | Loss 0.0028 | Win/lose count 7.0/5.0 (2.0)
Epoch 002/201 | Loss 0.0032 | Win/lose count 8.5/4.0 (4.5)
Epoch 003/201 | Loss 0.0291 | Win/lose count 6.0/9.0 (-3.0)
Epoch 004/201 | Loss 0.0208 | Win/lose count 6.0/8.0 (-2.0)
Epoch 005/201 | Loss 0.0104 | Win/lose count 8.0/5.0 (3.0)
Epoch 006/201 | Loss 0.0120 | Win/lose count 5.5/4.0 (1.5)
Epoch 007/201 | Loss 0.0141 | Win/lose count 9.0/6.0 (3.0)
Epoch 008/201 | Loss 0.0169 | Win/lose count 6.5/6.0 (0.5)
Epoch 009/201 | Loss 0.0095 | Win/lose count 8.0/3.0 (5.0)
Epoch 010/201 | Loss 0.0101 | Win/lose count 6.0/5.0 (1.0)
Epoch 011/201 | Loss 0.0154 | Win/lose count 7.5/4.0 (3.5)
Epoch 012/201 | Loss 0.0129 | Win/lose count 9.0/7.0 (2.0)
Epoch 013/201 | Loss 0.0138 | Win/lose count 7.5/2.0 (5.5)
Epoch 014/201 | Loss 0.0083 | Win/lose count 7.5/2.0 (5.5)
Epoch 015/201 | Loss 0.0152 | Win/lose count 20.0/5.0 (15.0)
Epoch 016/201 | Loss 0.0082 | Win/lose count 15.5/

Epoch 136/201 | Loss 0.0104 | Win/lose count 14.0/2.0 (12.0)
Epoch 137/201 | Loss 0.0219 | Win/lose count 12.5/3.0 (9.5)
Epoch 138/201 | Loss 0.0215 | Win/lose count 15.0/6.0 (9.0)
Epoch 139/201 | Loss 0.0199 | Win/lose count 9.0/5.0 (4.0)
Epoch 140/201 | Loss 0.0124 | Win/lose count 25.5/5.0 (20.5)
Epoch 141/201 | Loss 0.0553 | Win/lose count 22.0/7.0 (15.0)
Epoch 142/201 | Loss 0.0531 | Win/lose count 20.5/8.0 (12.5)
Epoch 143/201 | Loss 0.0201 | Win/lose count 15.5/4.0 (11.5)
Epoch 144/201 | Loss 0.0157 | Win/lose count 13.0/4.0 (9.0)
Epoch 145/201 | Loss 0.0385 | Win/lose count 22.5/2.0 (20.5)
Epoch 146/201 | Loss 0.0551 | Win/lose count 19.5/5.0 (14.5)
Epoch 147/201 | Loss 0.0616 | Win/lose count 17.5/3.0 (14.5)
Epoch 148/201 | Loss 0.0353 | Win/lose count 8.0/1.0 (7.0)
Epoch 149/201 | Loss 0.0723 | Win/lose count 15.5/11.0 (4.5)
Epoch 150/201 | Loss 0.0263 | Win/lose count 15.5/4.0 (11.5)
Epoch 151/201 | Loss 0.0259 | Win/lose count 23.0/9.0 (14.0)
Epoch 152/201 | Loss 0.0053 | W

In [21]:
# Evaluation
test_explore(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore20.mp4'))

Iteration 0. Win/lose count 20.0/5.0. Average score (15.0)
Iteration 1. Win/lose count 15.0/6.0. Average score (12.0)
Iteration 2. Win/lose count 3.5/2.0. Average score (8.5)
Iteration 3. Win/lose count 19.5/8.0. Average score (9.25)
Iteration 4. Win/lose count 24.5/5.0. Average score (11.3)
Iteration 5. Win/lose count 21.0/8.0. Average score (11.583333333333334)
Iteration 6. Win/lose count 23.0/9.0. Average score (11.928571428571429)
Iteration 7. Win/lose count 23.0/7.0. Average score (12.4375)
Iteration 8. Win/lose count 13.5/3.0. Average score (12.222222222222221)
Iteration 9. Win/lose count 17.0/5.0. Average score (12.2)
Iteration 10. Win/lose count 16.5/3.0. Average score (12.318181818181818)
Iteration 11. Win/lose count 15.5/3.0. Average score (12.333333333333334)
Iteration 12. Win/lose count 18.5/2.0. Average score (12.653846153846153)
Iteration 13. Win/lose count 8.0/4.0. Average score (12.035714285714286)
Iteration 14. Win/lose count 25.5/6.0. Average score (12.533333333333333

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***